# Lab Solutions: Web Scraping + File I/O

##### Instructions: 

1. Go to https://polisci.wustl.edu/people/88/all OR https://polisci.wustl.edu/people/list/88/all
2. Go to the page for each of the professors.
3. Create a `.csv`` file with the following information for each professor:
	- Name
	- Title
	- E-mail
	- Web page
	- Specialization  
		- If they do not have a specialization, you can leave it blank. 
		- An example from Deniz's page: https://polisci.wustl.edu/people/deniz-aksoy
		- Professor Aksoy’s research is motivated by an interest in comparative political institutions and political violence. 


In [38]:
from bs4 import BeautifulSoup
import urllib
import csv
import time

In [39]:
import sys
print(sys.executable)
# which python

import sys, importlib.util, pprint
print("python:", sys.executable)
print("version:", sys.version)
pprint.pprint(sys.path[:5])
print("selenium spec:", importlib.util.find_spec("selenium"))

/usr/local/bin/python3
python: /usr/local/bin/python3
version: 3.13.6 (v3.13.6:4e665351082, Aug  6 2025, 11:22:35) [Clang 16.0.0 (clang-1600.0.26.6)]
['/Library/Frameworks/Python.framework/Versions/3.13/lib/python313.zip',
 '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13',
 '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload',
 '',
 '/Users/riverjeon/Library/Python/3.13/lib/python/site-packages']
selenium spec: ModuleSpec(name='selenium', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1046a0290>, origin='/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/selenium/__init__.py', submodule_search_locations=['/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/selenium'])


In [40]:
from bs4 import BeautifulSoup
import urllib
import csv
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

In [41]:
os.getcwd()

'/Users/riverjeon/Documents/github/PythonCamp2025/Day02/Day02_Part02/Lab'

In [42]:
# os.chdir('/Users/almavelazquez/Documents/GitHub/PythonCamp2024/Day04/Lab')

In [43]:
with open('lab04.csv', 'w') as f: # open new csv file
    w = csv.DictWriter(f, fieldnames = ("name", "title", "email", "website", "specialization")) # set colnames
    w.writeheader() # write header

	# set up selenium stuff
    driver_path = Service('/Users/almavelazquez/Documents/GitHub/PythonCamp2024/Day04/Lecture/chromedriver')

	# open webpage
    driver = webdriver.Chrome(service = driver_path)
    driver.get('https://polisci.wustl.edu/people/88/all') # go to website
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # scroll to bottom
    time.sleep(5) # pause to let selenium catch up and actually scroll
    html = driver.page_source # get html
    driver.close() # close selenium browser
    soup = BeautifulSoup(html) # soup html

    cards = soup.find_all('a', {'class' : 'card'}) # get all faculty

    for c in range(len(cards)+1): # for each faculty card
        print("Working on " + str(c) + " of " + str(len(cards)) + ":")
        try:
            fac = {} # empty dict
            fac['name'] = ' '.join(cards[c].find('h3').text.split('\xa0')) # get name, split on weird encoding and re-join
            fac['title'] = cards[c].find('div', {'class' : 'dept'}).text # get title

            interior = 'https://polisci.wustl.edu' + cards[c]['href'] # go to interior page
            interior_page = urllib.request.urlopen(interior) # open interior
            interior_soup = BeautifulSoup(interior_page.read()) # soup interior

            fac['email'] = interior_soup.find('ul', {'class' : 'detail contact'}).find('a').text # get email
            fac['website'] = interior_soup.find('ul', {'class' : 'links'}).find('a')['href'] # get personal website
            fac['specialization'] = interior_soup.find('div', {'class' : 'post-excerpt'}).text # get specialization
            w.writerow(fac) # write row
        except:
            continue # skip row on issue (e.g., Lee Epstien) # NOT the best way to deal with this. 

        time.sleep(3) # polite sleep

print("All done!")

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
